# Metody Obliczeniowe w Nauce i Technice
## Laboratorium 3 - Singular Value Decomposition
### Albert Gierlach

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false; 
}

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mtplt
from PIL import Image

mtplt.rcParams['figure.figsize'] = [14, 9] # plots size
np.set_printoptions(precision=4)

#### Zadanie 1 -  Przekształcenie sfery w elipsoidę

###### 1.1 Sfera jednostkowa

In [ ]:
def draw_3d_plot(x,y,z, view=(10,20)):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=view[0], azim=view[1])
    ax.plot_surface(x,y,z, alpha=0.4)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    plt.show()


s = np.linspace(0, 2 * np.pi, 100)
t = np.linspace(0, np.pi, 100)

x = np.outer(np.cos(s), np.sin(t))
y = np.outer(np.sin(s), np.sin(t))
z = np.outer(np.ones(np.size(s)), np.cos(t))

draw_3d_plot(x,y,z)

##### 1.2 Przekształcenie sfery w elipsoide

In [ ]:
def transform_sphere(A):
    s = np.linspace(0, 2 * np.pi, 100)
    t = np.linspace(0, np.pi, 100)
    x = (np.outer(np.cos(s), np.sin(t)) * A.item(0, 0)) + \
        (np.outer(np.sin(s), np.sin(t)) * A.item(0, 1)) + \
        (np.outer(np.ones(np.size(s)), np.cos(t)) * A.item(0, 2))
    
    y = (np.outer(np.cos(s), np.sin(t)) * A.item(1, 0)) + \
        (np.outer(np.sin(s), np.sin(t)) * A.item(1, 1)) + \
        (np.outer(np.ones(np.size(s)), np.cos(t)) * A.item(1, 2))
    
    z = (np.outer(np.cos(s), np.sin(t)) * A.item(2, 0)) + \
        (np.outer(np.sin(s), np.sin(t)) * A.item(2, 1)) + \
        (np.outer(np.ones(np.size(s)), np.cos(t)) * A.item(2, 2))
    return x, y, z


A = []
A.append(np.array([[1,2,0],[1,1,1],[3,1,0]]))
A.append(np.array([[4,0,4],[0,3,0],[-3,0,3]]))
A.append(np.array([[5,0,1],[2,3,0],[0,0,4]]))

for transform_matrix in A:
    x1, y1, z1 = transform_sphere(transform_matrix)
    draw_3d_plot(x1,y1,z1)

##### 1.3 Rozkład według wartości osobliwych (SVD)

In [ ]:
for transform_matrix in A:
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=20, azim=20)

    U, S, VT = np.linalg.svd(transform_matrix)
    S_diag = np.diag(S) # because S is vector and we need diagonal matrix
    x1, y1, z1 = transform_sphere(transform_matrix)

    for s1_row in S_diag:
        base = np.dot(U, s1_row) # 3d base vec for one axis
        ax.plot([0, base[0]], [0, base[1]], [0, base[2]], linewidth=4)

    ax.plot_surface(x1,y1,z1,  alpha=0.3, color='r')
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    plt.show()

##### 1.4 Znalezienie macierzy, której stosunek największej wartości osobliwej i najmniejszej wynosi więcej niż 100

In [ ]:
def get_matirx_with_ratio_greater_than_100():
    while True:
        AA = np.random.rand(3, 3)
        U, S, VT = np.linalg.svd(AA)

        if S[0] / S[2] > 100:
            return AA
        
A100 = get_matirx_with_ratio_greater_than_100()
U, S, VT = np.linalg.svd(A100)
print("Znaleziona macierz:")
print(A100) # found matrix
print("\nWartości osobliwe tej macierzy")
print(np.diag(S)) # singular values
print("\nStosunek najwiekszej wartosci osobliwej do najmniejszej: {}".format(S[0] / S[2]))
x1, y1, z1 = transform_sphere(A100)
draw_3d_plot(x1,y1,z1)
draw_3d_plot(x1,y1,z1, (10,90)) # ellipses might be different, so generate three views
draw_3d_plot(x1,y1,z1, (90,80))

##### 1.5 Wizualizacje przekształceń dla wybranej macierzy
Wybrana macierz $ A_i = \left( \begin{matrix} 1 & 2 & 0 \\ 1 & 1 & 1 \\ 3 & 1 & 0 \end{matrix} \right) $\
Kolejne przekształcenia: $SV_i^T$, $S \Sigma_i V_i^T$, $S U_i \Sigma_i V_i^T$ \
Dla wizualizacji zaznaczono wektory kanoniczne - przed i po transformacji

In [ ]:
# helper functions to properly set axis limit
def check_one_limit(x, tuple_limits):
    a, b = abs(x.min()), abs(x.max())
    a = max(a, b)
    if a > tuple_limits[0]:
        return a

    return -1

def update_limits(ax, x,y,z, force=False):
    max_dim = max(check_one_limit(x, ax.get_xlim()), check_one_limit(y, ax.get_ylim()), check_one_limit(z, ax.get_zlim()))

    if max_dim != -1 or force:
        ax.set_xlim(max_dim, -max_dim)
        ax.set_ylim(max_dim, -max_dim)
        # ax.set_zlim(max_dim, -max_dim)

def transform_sphere_and_draw_vectors(A, ax):
    # translate canonical vector
    ident = np.identity(3)
    for v in range(3):
        res_v = A @ ident[v,:]
        ax.quiver(0, 0, 0, res_v[0], res_v[1], res_v[2], color='b', arrow_length_ratio=0.2)
        # ax.plot([0, res_v[0]], [0, res_v[1]], [0, res_v[2]], linewidth=2, color='b')

    return transform_sphere(A)

def visualize_transform(matrix, alpha=0.3, rstride=5, cstride=5):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.view_init(elev=30, azim=35)

    # base sphere
    x, y, z = transform_sphere(np.identity(3))
    ax.plot_wireframe(x, y, z, alpha=alpha, color='g', rstride=rstride, cstride=cstride)
    update_limits(ax, x, y, z)

    x, y, z = transform_sphere_and_draw_vectors(matrix, ax)
    ax.plot_wireframe(x, y, z, alpha=alpha, color='r', rstride=rstride, cstride=cstride)
    update_limits(ax, x, y, z)

    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    plt.show()

In [ ]:
M = A[0] # our matrix
U, S, VT = np.linalg.svd(M)
S = np.diag(S)

In [ ]:
visualize_transform(VT)

In [ ]:
visualize_transform(S.dot(VT))

In [ ]:
visualize_transform(U.dot(S).dot(VT))

In [ ]:
visualize_transform(M)

### Wnioski
Pierwsze przekształcenie sfery jednostkowej pokazuje, że transformacja za pomocą macierzy ortogonalnej $V^T$ pozwala na obracanie obiektu wzdłuż osi $OX$. Drugie przekształcenie w połączeniu z pierwszym pozwala na skalowanie obiektu. Dzieje się tak dlatego, gdyż macierz $S$ jest diagonalna, więc jest po prostu czynnikiem skalującym. Ostatnie dwa przekształcenia są sobie równoważne. Mnożenie przez macierz $U$ także powoduje obrót figury.

### Zadanie 2 - Kompresja obrazu

#### 1.1 Przygotowanie przykładowego zdjęcia

<img src="./img.bmp">

#### 1.2 Low rank approximation

##### 1.2.1 Skala szarości

In [ ]:
filename = "./img.jpg"
img = Image.open(filename).convert('L')
A = np.array(img)
U, S, VT = np.linalg.svd(A)

per_row = 3
iter_num = 39
for k in range(min(A.shape[0], iter_num)):
    I = U[:, :k] @ np.diag(S[:k]) @ VT[:k, :]
    
    if k % per_row == 0 and k:
        plt.show()

    plt.subplot(1, per_row, (k % per_row) + 1)
    plt.title('k={}'.format(k))
    plt.axis('off')
    plt.imshow(I, cmap='gray')
plt.show()

##### 1.2.2 Kolor

In [ ]:
filename = "./img.jpg"
img = Image.open(filename)
w, h = img.width, img.height
A = np.array(img)
U, S, VT, I_colors = [None] * 3, [None] * 3, [None] * 3, [None] * 3
for i in range(3):
    U[i], S[i], VT[i] = np.linalg.svd(A[:, :, i])

per_row = 3
iter_num = 39
for k in range(min(A.shape[0], iter_num)):
    for i in range(3):
        # 0 - r, 1 - g, 2 - b
        I_colors[i] = U[i][:, :k] @ np.diag(S[i][:k]) @ VT[i][:k, :]
        I_colors[i].reshape(w * h)

    I = np.asarray([I_colors[0], I_colors[1], I_colors[2]]).transpose((1, 2, 0)).reshape(h, w, 3)
    img2 = Image.fromarray(np.uint8(I))
    
    if k % per_row == 0 and k:
        plt.show()

    plt.subplot(1, per_row, (k % per_row) + 1)
    plt.title('k={}'.format(k))
    plt.axis('off')
    plt.imshow(img2)
plt.show()

#### 1.3 Porównanie kilku obrazów wynikowych z obrazem oryginalnym

##### 1.3.1 Skala szarości

In [ ]:
filename = "./img.jpg"
img = Image.open(filename).convert('L')
A = np.array(img)
U, S, VT = np.linalg.svd(A)

ranks = [1, 10, 20, 30, 40, 60, 100, 200]
for i in range(min(A.shape[0], len(ranks))):
    k = ranks[i]
    I = U[:, :k] @ np.diag(S[:k]) @ VT[:k, :]

    plt.subplot(1, 2, 1)
    plt.title('k={}'.format(k))
    plt.axis('off')
    plt.imshow(I, cmap='gray')
        
    plt.subplot(1, 2, 2)
    plt.title('original')
    plt.axis('off')
    plt.imshow(img, cmap='gray')

    plt.show()

##### 1.3.2 Kolor

In [ ]:
filename = "./img.jpg"
img = Image.open(filename)
w, h = img.width, img.height
A = np.array(img)
U, S, VT, I_colors = [None] * 3, [None] * 3, [None] * 3, [None] * 3
for i in range(3):
    U[i], S[i], VT[i] = np.linalg.svd(A[:, :, i])

ranks = [1, 10, 20, 30, 40, 60, 100, 200, 400]
for ii in range(min(A.shape[0], len(ranks))):
    k = ranks[ii]
    for i in range(3):
        # 0 - r, 1 - g, 2 - b
        I_colors[i] = U[i][:, :k] @ np.diag(S[i][:k]) @ VT[i][:k, :]
        I_colors[i].reshape(w * h)

    I = np.asarray([I_colors[0], I_colors[1], I_colors[2]]).transpose((1, 2, 0)).reshape(h, w, 3)
    img2 = Image.fromarray(np.uint8(I))

    plt.subplot(1, 2, 1)
    plt.title('k={}'.format(k))
    plt.axis('off')
    plt.imshow(img2)

    plt.subplot(1, 2, 2)
    plt.title('original')
    plt.axis('off')
    plt.imshow(img)

    plt.show()

### Wnioski
Przedstawione serie wygenerowanych obrazów pokazują, że SVD może zostać użyte do kompresji obrazów. Dla obrazu w skali szarości uzyskano najlepsze rezultaty, bo już przy k = 30 dało się dość dobrze rozpoznać detale obrazu. Gdy weźmiemy pod uwagę zdjęcie kolorowe to zauważymy, że przy stosunkowo niskich k (np. k < 50) występują pojedyncze artefakty, ale po mimo tego obraz jest bardzo czytelny.

#### Źródła:
* Numerical Mathematics and Computing, 7th Edition - Ward Cheney, David Kincaid
* [Deep Learning Book Series · 2.8 Singular Value Decomposition](https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.8-Singular-Value-Decomposition/)
* [Związek analizy korespondencji z tabelą krzyżową – skąd się biorą współrzędne punktów?](https://predictivesolutions.pl/zwiazek-analizy-korespondencji-z-tabela-krzyzowa-skad-sie-biora-wspolrzedne-punktow)
* [Rozkład według wartości osobliwych - Singular value decomposition](https://pl.qwe.wiki/wiki/Singular_value_decomposition#Singular_values_as_semiaxis_of_an_ellipse_or_ellipsoid)